In [1]:
import os

from dotenv import load_dotenv
from etg import GuestRoom

load_dotenv()

# ETG API Credentials
ETG_KEY_ID = os.environ["ETG_KEY_ID"]
ETG_API_KEY = os.environ["ETG_API_KEY"]
ETG_REQUEST_TIMEOUT = 30.0

# Search Parameters
CITY = "Москва"

CHECKIN_DATE = "2026-02-02"
CHECKOUT_DATE = "2026-02-04"

CURRENCY = "RUB"
LANGUAGE = "ru"
RESIDENCY = "RU"

GUESTS: list[GuestRoom] = [{"adults": 2, "children": [4, 2]}]
LIMIT = 1000

# User preferences for AI
USER_PREFERENCES = "Обязательно две комнаты и две кровати. Хорошие отзывы. Чистота"

# Filters
MIN_PRICE: float | None = 3000.0  # None = no minimum
MAX_PRICE: float | None = 20000.0  # None = no maximum

In [2]:
from etg import ETGClient, GuestRoom, Hotel, HotelContent
from utils import ostrovok_url

client = ETGClient(ETG_KEY_ID, ETG_API_KEY, timeout=ETG_REQUEST_TIMEOUT)

In [3]:
async def find_region_id(client: ETGClient, city_name: str, language: str) -> int | None:
    """Find region ID for a city name."""
    print(f"Looking up region ID for '{city_name}'...")
    regions = await client.suggest_region(city_name, language)

    if not regions:
        print(f"  No regions found for '{city_name}'")
        return None

    # Only accept City type
    for region in regions:
        if region["type"] == "City":
            region_id = region["id"]
            print(f"  Found: {region['name']} ({region.get('country_code', '')}), region_id={region_id}")
            return region_id

    # No city found - show available options
    print(f"  No city found. Available regions:")
    for r in regions[:5]:
        print(f"    - {r['name']} (type: {r['type']}, id: {r['id']})")

    return None

In [4]:
# Find region by city name.
# ETG API requires region_id for hotel search, so we first
# lookup the region ID via suggest_region by city name.
region_id = await find_region_id(client, CITY, LANGUAGE)
if not region_id:
    raise ValueError(f"Could not find region for '{CITY}'")

print(f"\nSearching hotels in {CITY}...")
print(f"  Dates: {CHECKIN_DATE} to {CHECKOUT_DATE}")
print(f"  Currency: {CURRENCY}, Limit: {LIMIT}")

Looking up region ID for 'Москва'...
  Found: Москва (RU), region_id=2395

Searching hotels in Москва...
  Dates: 2026-02-02 to 2026-02-04
  Currency: RUB, Limit: 1000


In [5]:
import pandas as pd

from services import filter_hotels_by_price

# Search available hotels in the region with given parameters.
# Filters by price range if MIN_PRICE/MAX_PRICE are set.
# Returns short hotel info: id, hid, and rates (room name, price, meal).
# Full content (name, address, amenities) is fetched separately via hotel content API.
# Search available hotels in the region with given parameters.
# Filters by price range if MIN_PRICE/MAX_PRICE are set.
# Returns short hotel info: id, hid, and rates (room name, price, meal).
# Full content (name, address, amenities) is fetched separately via hotel content API.
search_results = await client.search_hotels_by_region(
    region_id=region_id,
    checkin=CHECKIN_DATE,
    checkout=CHECKOUT_DATE,
    residency=RESIDENCY,
    guests=GUESTS,
    currency=CURRENCY,
    language=LANGUAGE,
    hotels_limit=LIMIT,
)

all_hotels = search_results.get("hotels", [])
total_available = search_results.get("total_hotels", len(all_hotels))

# Filter by price
hotels = filter_hotels_by_price(all_hotels, MIN_PRICE, MAX_PRICE)
total_after_filter = len(hotels)

In [6]:
if not hotels:
    print("No hotels found for the given criteria.")
    df_hotels = pd.DataFrame()
else:
    print(f"Found {total_after_filter} hotels after price filter (total available: {total_available})\n")

    # Create DataFrame
    hotels_data = []
    for hotel in hotels:
        rates = hotel.get("rates", [])
        if rates:
            first_rate = rates[0]
            payment_types = first_rate.get("payment_options", {}).get("payment_types", [])
            price = float(payment_types[0].get("show_amount", 0)) if payment_types else None
            room_name = first_rate.get("room_name", "")
            meal = first_rate.get("meal", "")
        else:
            price = None
            room_name = ""
            meal = ""

        hotels_data.append({
            "hotel_id": hotel["id"],
            "hid": hotel["hid"],
            "price": price,
            "currency": CURRENCY,
            "room": room_name[:40] if room_name else "",
            "meal": meal,
        })

    df_hotels = pd.DataFrame(hotels_data)
    df_hotels = df_hotels.sort_values("price", ascending=True).reset_index(drop=True)
    df_hotels.index += 1  # Start from 1

    # Display settings
    pd.set_option("display.max_colwidth", 50)
    pd.set_option("display.max_rows", 100)

df_hotels

Found 299 hotels after price filter (total available: 328)



,hotel_id,hid,price,currency,room,meal
1,na_baumanskoy_hotel,10613383,6074.0,RUB,Четырёхместный номер семейный Эконом-кла,nomeal
2,metro_apartments_5,9990817,6889.0,RUB,Четырёхместный номер Комфорт Двухуровнев,nomeal
3,travel_inn_krasnyie_vorota,8877227,6951.0,RUB,Семейный номер Economy (общая ванная ком,nomeal
4,apartamentyi_sadovoe_koltso_izumrudnaya,8510363,6970.0,RUB,Апартаменты (питание для детей не включе,nomeal
5,zvezda_hotel_6,10437016,7000.0,RUB,Трёхместный номер Comfort (дополнительна,nomeal
...,...,...,...,...,...,...
295,moscow_marriott_hotel_novy_arbat,8139930,38000.0,RUB,Двухместный номер Делюкс (2 отдельные кр,nomeal
296,russkie_sezonyi,8848163,38106.0,RUB,Двухместный люкс with Sofa Superior Манс,nomeal
297,hampton_by_hilton_moscow_rogozhsky_val,9770069,38522.0,RUB,Номер с диваном Семейный с кроватью King,breakfast
298,grand_revival_hotel,10004948,38836.0,RUB,Четырёхместный номер Большой Улучшенный,nomeal


In [8]:
from services import batch_get_content

hotel_hids = [h["hid"] for h in hotels]

print(f"[batch_get_content_start] Загрузка контента для {len(hotel_hids)} отелей...")
content_map = await batch_get_content(client, hotel_hids, LANGUAGE)
print(f"[batch_get_content_done] Загружен контент для {len(content_map)} из {len(hotel_hids)} отелей")

[batch_get_content_start] Загрузка контента для 299 отелей...
[batch_get_content_done] Загружен контент для 299 из 299 отелей


In [9]:
content_data = []
for hid, content in content_map.items():
    content_data.append({
        "hid": hid,
        "name": content.get("name", "")[:40],
        "stars": content.get("star_rating", 0),
        "kind": content.get("kind", ""),
        "address": content.get("address", "")[:50],
        "latitude": content.get("latitude"),
        "longitude": content.get("longitude"),
        "check_in": content.get("check_in_time", ""),
        "check_out": content.get("check_out_time", ""),
    })

df_content = pd.DataFrame(content_data)

# Merge with prices from df_hotels
df_full = df_hotels.merge(df_content, on="hid", how="left")
df_full = df_full[["hotel_id", "name", "stars", "kind", "price", "meal", "address"]]
df_full = df_full.sort_values("price", ascending=True).reset_index(drop=True)
df_full.index += 1

df_full

,hotel_id,name,stars,kind,price,meal,address
1,na_baumanskoy_hotel,Отель на Бауманской,0,Hotel,6074.0,nomeal,"улица Фридриха Энгельса, д.31/35, Москва"
2,metro_apartments_5,Апарт-отель Metro rooms,3,Apart-hotel,6889.0,nomeal,"проезд Гостиничный, д.6, корп.2, Москва"
3,travel_inn_krasnyie_vorota,Мини-отель на Комсомольской,0,Mini-hotel,6951.0,nomeal,"улица Новая Басманная, д.10/1, Москва"
4,apartamentyi_sadovoe_koltso_izumrudnaya,Апартаменты Садовое Кольцо Изумрудная,0,Apartment,6970.0,nomeal,"Изумрудная улица, д.6, Москва"
5,zvezda_hotel_6,Городской отель Звезда,0,Hotel,7000.0,nomeal,"г. Люберцы ул. 3-е Почтовое Отделение, д. 44А,..."
...,...,...,...,...,...,...,...
295,moscow_marriott_hotel_novy_arbat,Отель Звёзды Арбата (ранее Мариотт Новый,5,Hotel,38000.0,nomeal,"ул. Новый Арбат, 32, Москва, Москва"
296,russkie_sezonyi,Бутик-Отель Регул,5,Boutique_and_Design,38106.0,nomeal,"Дегтярный переулок, д. 8/2, Москва"
297,hampton_by_hilton_moscow_rogozhsky_val,Отель Hampton by Hilton Moscow Рогожский,4,Hotel,38522.0,breakfast,"улица Рогожский Вал, дом 10, Москва"
298,grand_revival_hotel,Отель Revival Hotel,3,Hotel,38836.0,nomeal,"улица Петровка 19 стр.3, Москва"


In [10]:
from services import batch_get_reviews, filter_reviews, HotelReviews

print(f"[batch_get_reviews_start] Загрузка отзывов для {len(hotel_hids)} отелей...")
raw_reviews = await batch_get_reviews(client, hotel_hids, LANGUAGE)
reviews_map = filter_reviews(raw_reviews)

total_raw = sum(rd["total_reviews"] for rd in raw_reviews.values())
total_filtered = sum(len(rd["reviews"]) for rd in reviews_map.values())
# Compute filtered_by_age: how many reviews were filtered out
total_raw_in_filtered = sum(rd["total_reviews"] for rd in reviews_map.values())
total_filtered_by_age = total_raw_in_filtered - total_filtered
hotels_with_reviews = len(reviews_map)

# Calculate average rating across all hotels
all_avg_ratings = [rd["avg_rating"] for rd in reviews_map.values() if rd["avg_rating"] is not None]
overall_avg = sum(all_avg_ratings) / len(all_avg_ratings) if all_avg_ratings else 0

print(
    f"[batch_get_reviews_done] Всего {hotels_with_reviews} отелей с отзывами из {len(hotel_hids)}"
)
print(
    f"  Обработано {total_raw} отзывов → {total_filtered} релевантных "
    f"(отсечено по давности: {total_filtered_by_age})"
)
print(f"  Средний рейтинг: {overall_avg:.1f}/10")

[batch_get_reviews_start] Загрузка отзывов для 299 отелей...
[batch_get_reviews_done] Всего 296 отелей с отзывами из 299
  Обработано 47115 отзывов → 37119 релевантных (отсечено по давности: 9996)
  Средний рейтинг: 8.5/10


In [11]:
# Create DataFrame with reviews summary
reviews_data = []
for hid, data in reviews_map.items():
    hotel_id = next((h["id"] for h in hotels if h["hid"] == hid), "")
    detailed = data["detailed_averages"]
    reviews_data.append({
        "hotel_id": hotel_id,
        "hid": hid,
        "total": data["total_reviews"],
        "avg_rating": data["avg_rating"],
        "cleanness": detailed["cleanness"],
        "location": detailed["location"],
        "price": detailed["price"],
        "services": detailed["services"],
        "room": detailed["room"],
        "meal": detailed["meal"],
        "wifi": detailed["wifi"],
        "hygiene": detailed["hygiene"],
    })

df_reviews = pd.DataFrame(reviews_data)
df_reviews = df_reviews.sort_values("avg_rating", ascending=False, na_position="last").reset_index(drop=True)
df_reviews.index += 1

# Merge with hotel info
df_reviews_full = df_reviews.merge(
    df_content[["hid", "name", "stars"]],
    on="hid",
    how="left"
)
df_reviews_full = df_reviews_full[[
    "hotel_id", "name", "stars", "total", "avg_rating",
    "cleanness", "location", "room", "services", "price", "meal", "wifi", "hygiene"
]]


def show_reviews(hotel_id: str, limit: int = 5) -> None:
    """Show reviews for a hotel."""
    hid = next((h["hid"] for h in hotels if h["id"] == hotel_id), None)
    if not hid:
        print(f"Hotel '{hotel_id}' not found")
        return
    
    data = reviews_map.get(hid)
    if not data:
        print(f"No reviews for hotel '{hotel_id}'")
        return
    
    hotel_name = content_map.get(hid, {}).get("name", hotel_id)
    avg = data["avg_rating"]
    d = data["detailed_averages"]
    
    print(f"{'='*70}")
    print(f"{hotel_name}")
    print(f"Total: {data['total_reviews']} reviews | Avg rating: {avg}/10" if avg else f"Total: {data['total_reviews']} reviews")
    print(f"{'='*70}")
    print(f"Detailed scores:")
    print(f"  cleanness: {d['cleanness']}  location: {d['location']}  room: {d['room']}  services: {d['services']}")
    print(f"  price: {d['price']}  meal: {d['meal']}  wifi: {d['wifi']}  hygiene: {d['hygiene']}")
    print(f"{'='*70}\n")
    
    reviews = data["reviews"][:limit]
    for r in reviews:
        rating = r["rating"]
        date = r["created"][:10]
        lang = r.get("_lang", "?")
        plus = r.get("review_plus", "").strip()
        minus = r.get("review_minus", "").strip()
        print(f"[{rating}/10] {date} [{lang}]")
        if plus:
            print(f"  + {plus[:300]}")
        if minus:
            print(f"  - {minus[:300]}")
        print()


df_reviews_full

,hotel_id,name,stars,total,avg_rating,cleanness,location,room,services,price,meal,wifi,hygiene
0,osennyaya_apartments,Апартаменты Осенняя,0,4,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
1,volgogradskiy_pospekt_325_k3_apartments,Апартаменты Волгоградский Проспект 32/5,0,7,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
2,great_apart_na_arbate_1,Great apart Duplex на Арбате,0,1,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
3,vmestimostyyu_do_5_gostey_ryadom_s_metro_ot_ts...,Квартира Вместимостью до 5 Гостей Рядом,0,1,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
4,bobolink_cold_na_sokolinoy_gore_lodging_house,Апартаменты двухуровневые Cosmo Star на,0,10,10.0,10.0,10.0,10.0,10.0,10.0,NaN,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,ladomir_v_filyah,Отель Мини-отель Ладомир-отель,0,26,4.9,4.5,5.6,3.9,5.1,5.2,2.6,10.0,2.0
292,zvezda_hotel_6,Городской отель Звезда,0,96,4.8,4.6,5.1,4.0,5.0,4.9,6.2,10.0,5.0
293,hotel_apelsin_vnukovo,Апельсин Гостевой Дом во Внуково,0,35,4.4,4.1,5.9,3.6,4.0,3.8,4.8,10.0,NaN
294,moskva_siti_apex_42_etazh_2_komnaty_apartments,Апартаменты Москва Сити Apex 42 этаж 2 к,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Example: view reviews for a specific hotel
# show_reviews("four_seasons_st_petersburg")       # 5 reviews
# show_reviews("four_seasons_st_petersburg", 10)   # 10 reviews

show_reviews("four_seasons_st_petersburg", limit=3)

Hotel 'four_seasons_st_petersburg' not found


In [13]:
from services import combine_hotels_data, HotelFull

combined = combine_hotels_data(hotels, content_map, reviews_map)
print(f"Combined {len(combined)} hotels with content and reviews")

Combined 299 hotels with content and reviews


In [14]:
import json

from services import estimate_tokens, prepare_hotel_for_llm, presort_hotels, score_hotels

# Estimate tokens before presort
hotels_for_llm_all = [prepare_hotel_for_llm(h) for h in combined]
tokens_before = estimate_tokens(json.dumps(hotels_for_llm_all, ensure_ascii=False))

# Pre-sort by hotel kind tier and prescore, limit to top 100 for LLM scoring
PRESORT_LIMIT = 300
top_hotels = presort_hotels(combined, reviews_map, limit=PRESORT_LIMIT)

# Estimate tokens after presort
hotels_for_llm_top = [prepare_hotel_for_llm(h) for h in top_hotels]
tokens_after = estimate_tokens(json.dumps(hotels_for_llm_top, ensure_ascii=False))

print(f"[presort_done] {len(combined)} отелей → {len(top_hotels)} (лимит {PRESORT_LIMIT})")
print(f"  Токены: ~{tokens_before:,} → ~{tokens_after:,} (экономия {tokens_before - tokens_after:,})")

[presort_done] 299 отелей → 299 (лимит 300)
  Токены: ~271,481 → ~271,481 (экономия 0)


In [15]:
import time

from services import finalize_scored_hotels, score_hotels

# Score hotels using single LLM request
# Returns top 10 scored hotels with summary explaining the selection
print(f"[scoring_start] Scoring {len(top_hotels)} hotels...")
start_time = time.time()

scoring_result = await score_hotels(
    top_hotels,
    USER_PREFERENCES,
    guests=GUESTS,
    min_price=MIN_PRICE,
    max_price=MAX_PRICE,
    currency=CURRENCY,
    top_count=10
)

elapsed = time.time() - start_time

if scoring_result["error"]:
    print(f"\n❌ ERROR: {scoring_result['error']}")
    scored_hotels = None
else:
    scoring_results = scoring_result["results"]
    print(f"[scoring_done] {len(scoring_results)} hotels scored — {elapsed:.1f}s")
    print(f"  Estimated tokens: ~{scoring_result['estimated_tokens']:,}")
    print(f"\n📋 Summary:\n{scoring_result['summary']}")
    
    # Finalize scored hotels - merge scoring results with full hotel data
    scored_hotels = finalize_scored_hotels(combined, scoring_results)
    print(f"\n[finalize_done] {len(scored_hotels)} hotels with complete data")

[scoring_start] Scoring 299 hotels...
[scoring_done] 10 hotels scored — 39.3s
  Estimated tokens: ~273,441

📋 Summary:
The Moscow accommodation market for families is diverse, with a median price of approximately 12,000 RUB per night for quality stays. We analyzed a wide range of options from 3,000 to 20,000 RUB, focusing on properties that could accommodate two adults and two children while meeting the strict requirement for multiple rooms or high-capacity suites. The selection prioritized Tier 1 and 2 properties with guest ratings above 8.5 to ensure cleanliness and comfort. Properties like 'Hotel Voshod' (ID: hotel_voshod, rating 6.8/10) and 'Hotel Soblaznov' (ID: hotel_soblaznov, rating 6.6/10) were excluded due to low guest satisfaction and poor value compared to higher-rated alternatives. Our final recommendations emphasize properties that offer the best balance of space, high star ratings, and exceptional guest feedback.

[finalize_done] 10 hotels with complete data


In [16]:
from services import HotelScored


def display_top_hotels(
    scored_hotels: list[HotelScored],
    top_n: int = 10,
) -> pd.DataFrame:
    """Display top N scored hotels with details and Ostrovok links."""
    print(f"\n{'='*80}")
    print(f"TOP {top_n} HOTELS")
    print(f"{'='*80}\n")

    data = []
    for i, hotel in enumerate(scored_hotels[:top_n], 1):
        hotel_id = hotel["id"]
        name = hotel["name"]
        hid = hotel["hid"]
        kind = hotel.get("kind", "")
        score = hotel["score"]
        reasons = hotel.get("top_reasons", [])
        penalties = hotel.get("score_penalties", [])
        selected_hash = hotel.get("selected_rate_hash")
        
        # Find selected rate by hash
        rates = hotel.get("rates", [])
        selected_rate = next((r for r in rates if r.get("match_hash") == selected_hash), None)
        
        # Get rate details
        if selected_rate:
            room_name = selected_rate.get("room_name", "")[:50]
            meal_data = selected_rate.get("meal_data", {})
            meal = meal_data.get("value", selected_rate.get("meal", ""))
            
            # Calculate prices from daily_prices
            daily_prices = selected_rate.get("daily_prices", [])
            if daily_prices:
                # Convert string prices to float and sum
                total_price = sum(float(p) for p in daily_prices)
                num_nights = len(daily_prices)
                avg_price_per_night = total_price / num_nights if num_nights > 0 else 0
                
                # Get currency
                pt = selected_rate.get("payment_options", {}).get("payment_types", [])
                currency = pt[0].get("show_currency_code", "") if pt else ""
                
                total_price_str = f"{total_price:.0f} {currency}"
                avg_price_str = f"{avg_price_per_night:.0f} {currency}"
            else:
                # Fallback to payment_types if daily_prices not available
                pt = selected_rate.get("payment_options", {}).get("payment_types", [])
                if pt:
                    total_price = float(pt[0].get("show_amount", 0))
                    currency = pt[0].get("show_currency_code", "")
                    total_price_str = f"{total_price:.0f} {currency}"
                    avg_price_str = f"{total_price:.0f} {currency}"
                else:
                    total_price_str = "N/A"
                    avg_price_str = "N/A"
        else:
            room_name = "N/A"
            meal = "N/A"
            total_price_str = "N/A"
            avg_price_str = "N/A"
        
        # Get reviews data
        reviews = hotel.get("reviews")
        avg_rating = reviews.get("avg_rating") if reviews else None
        detailed = reviews.get("detailed_averages", {}) if reviews else {}
        
        # Generate Ostrovok URL
        url = ostrovok_url(
            hotel_id=hotel_id,
            hid=hid,
            checkin=CHECKIN_DATE,
            checkout=CHECKOUT_DATE,
            guests=GUESTS,
            region_id=region_id,
        )

        # Print detailed info
        print(f"{i}. {name} [{kind}]")
        print(f"   Score: {score}/100 | Rating: {avg_rating}/10" if avg_rating else f"   Score: {score}/100")
        print(f"   Room: {room_name}")
        print(f"   Total: {total_price_str} | Avg per night: {avg_price_str} | Meal: {meal}")
        if reasons:
            print(f"   + {'; '.join(reasons[:3])}")
        if penalties:
            print(f"   - {'; '.join(penalties[:5])}")
        print(f"   🔗 {url}")
        print()
        
        # Collect for DataFrame
        data.append({
            "name": name[:35],
            "kind": kind,
            "room": room_name[:30],
            "total": total_price_str,
            "avg/night": avg_price_str,
            "meal": meal,
            "score": score,
            "rating": avg_rating,
            "clean": detailed.get("cleanness"),
            "url": url,
        })
    
    df = pd.DataFrame(data)
    df.index = range(1, len(df) + 1)
    selected = min(top_n, len(scored_hotels))
    print(f"Всего найдено {len(combined)} отелей на эти даты.")
    print(f"Подобраны лучшие {selected} по вашим критериям.")
    return df


pd.set_option("display.max_colwidth", 100)
display_top_hotels(scored_hotels, top_n=10)


TOP 10 HOTELS

1. Городской отель Дом Купца Бавыкина [Hotel]
   Score: 98/100 | Rating: 9.6/10
   Room: N/A
   Total: N/A | Avg per night: N/A | Meal: N/A
   + Perfectly matches the 'two rooms' requirement with a family suite; Exceptional guest rating (9.6/10) highlighting cleanliness and service; Great value for a unique historical property in a quiet location
   🔗 https://ostrovok.ru/hotel/russia/moscow/mid11347687/dom_kuptsa_bavykina_mini_hotel/?dates=02.02.2026-04.02.2026&guests=2and4.2&q=2395

2. Отель Кунь Лунь [Hotel]
   Score: 95/100 | Rating: 8.8/10
   Room: Двухместный люкс с 2 комнатами (двуспальная кроват
   Total: 15342 RUB | Avg per night: 7671 RUB | Meal: nomeal
   + Spacious 2-room suite explicitly matching user preferences; High-tier 4-Star hotel quality for a very competitive price; Strong guest rating (8.8/10) with positive feedback on room size
   - Location is 19km from the city center
   🔗 https://ostrovok.ru/hotel/russia/moscow/mid10013909/kunlun/?dates=02.02.20

,name,kind,room,total,avg/night,meal,score,rating,clean,url
1,Городской отель Дом Купца Бавыкина,Hotel,N/A,N/A,N/A,N/A,98,9.6,9.8,https://ostrovok.ru/hotel/russia/moscow/mid11347687/dom_kuptsa_bavykina_mini_hotel/?dates=02.02....
2,Отель Кунь Лунь,Hotel,Двухместный люкс с 2 комнатами,15342 RUB,7671 RUB,nomeal,95,8.8,9.2,https://ostrovok.ru/hotel/russia/moscow/mid10013909/kunlun/?dates=02.02.2026-04.02.2026&guests=2...
3,Отель Веллион Водный,Hotel,Апартаменты с 3 комнатами (пит,19490 RUB,9745 RUB,nomeal,92,8.5,8.6,https://ostrovok.ru/hotel/russia/moscow/mid8854273/wellion_vodny_hotel/?dates=02.02.2026-04.02.2...
4,Отель Державный,Hotel,Двухместный люкс с 2 комнатами,19121 RUB,9560 RUB,nomeal,90,8.8,8.8,https://ostrovok.ru/hotel/russia/moscow/mid8874860/derzhavnyij_hotel/?dates=02.02.2026-04.02.202...
5,Отель NORKA,Hotel,Четырёхместный семейный номер,7872 RUB,3936 RUB,nomeal,88,9.1,9.0,https://ostrovok.ru/hotel/russia/moscow/mid11227637/norka_aparthotel/?dates=02.02.2026-04.02.202...
6,Гостиница Арена,Hotel,Четырёхместный номер Standard,18267 RUB,9134 RUB,nomeal,85,7.8,7.3,https://ostrovok.ru/hotel/russia/moscow/mid7657481/gostinitsa_arena/?dates=02.02.2026-04.02.2026...
7,Гостиница Рослеспром,Hotel,Четырёхместный номер Deluxe с,13572 RUB,6786 RUB,breakfast,82,7.2,7.1,https://ostrovok.ru/hotel/russia/moscow/mid8059711/gostinitsa_roslesprom/?dates=02.02.2026-04.02...
8,Отель Мансарда,Hotel,Четырёхместный номер семейный,18523 RUB,9262 RUB,nomeal,80,7.8,7.9,https://ostrovok.ru/hotel/russia/moscow/mid8742042/mansarda_hotel/?dates=02.02.2026-04.02.2026&g...
9,Отель Авион Внуково,Hotel,Четырёхместный номер Deluxe с,18300 RUB,9150 RUB,nomeal,78,9.2,9.4,https://ostrovok.ru/hotel/russia/moscow/mid6426482/vnukovo_otel_green_palace/?dates=02.02.2026-0...
10,Отель WineWood Moscow,Hotel,Двухместный номер Делюкс (двус,18000 RUB,9000 RUB,nomeal,75,8.9,8.9,https://ostrovok.ru/hotel/russia/moscow/mid8854262/winewood_moscow_lounge_hotel/?dates=02.02.202...
